# 교차 검증과 그리드 서치

## 검증 세트

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
# wine의 attribute(=feature) 개수는 4개(도수, 당도, 산도, 와인종류)
# class = 0(red), 1(white)

# Decision Tree를 잘 나누는 gini index

print(wine)
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

      alcohol  sugar    pH  class
0         9.4    1.9  3.51    0.0
1         9.8    2.6  3.20    0.0
2         9.8    2.3  3.26    0.0
3         9.8    1.9  3.16    0.0
4         9.4    1.9  3.51    0.0
...       ...    ...   ...    ...
6492     11.2    1.6  3.27    1.0
6493      9.6    8.0  3.15    1.0
6494      9.4    1.2  2.99    1.0
6495     12.8    1.1  3.34    1.0
6496     11.8    0.8  3.26    1.0

[6497 rows x 4 columns]


In [ ]:
# Decision Tree를 나누는 train data 80%, test data 20%
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
# train data = sub + test(20%), test data = test(80%)
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
# fit()을 통해 학습 데이터 = 훈련 데이터 -> 학습시킴
dt.fit(sub_input, sub_target)

# overfitting : 모델이 훈련 데이터에서 더 잘 맞음
# DT에 아무 것도 안한 결과(match) = 86.4
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증 Cross validation

In [ ]:
# train data를 k개로 나누고 일부를 train data, 나머지를 test data로 모델을 학습 * 반복하여 훈련
# 각 결과의 평균을 통해 최종 결과를 만듬
# 일반화를 잘 시킴
from sklearn.model_selection import cross_validate

# test score 5개
# Cross validation 결과(match) = 86.9, 84,6..
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00986075, 0.00669026, 0.00681138, 0.00663424, 0.00639987]), 'score_time': array([0.00091052, 0.00087333, 0.00083447, 0.00079894, 0.00080872]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# StratifiedKFold : train data를 k개를 랜덤하게 섞음
# 현재 데이터의 경우, white wine보다 red wine의 수가 훨씬 적음 => 랜덤하게 섞음
from sklearn.model_selection import StratifiedKFold

# Cross validation 결과(match) = 85.5
# cross_validate의 기본이 StratifiedKFold()를 사용하므로 결과가 비슷함(예측 결과가 올라가지 않음)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# Cross validation 결과(match) = 85.7 (0.2 성능 향상!)
# 버킷의 개수를 10개로 늘림(n_splits)

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [ ]:
# DT의 성능 조절, overfitting을 줄임
# 1. depth 조절(max_depth) : depth를 중간에 잘라서 train 데이터에서는 성능이 떨어지지만, test 데이터에서는 성능이 올라감
# 2. 샘플 갯수 조절(min_sample) : 노드에 속해있는 샘플의 개수를 줄임 
# 3. gini impurity : 분순도가 낮으면 해당 노드는 더 이상 쪼개지 않음

# 하이퍼파라미터 튜닝 1) GridSearchCV : 격자를 통해 min_impurity 를 증가시켜서 각 성능을 확인
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# n_jobs = CPU의 core 성능을 조절 : -1 의 경우 연산장치를 전부 다 사용하겠다는 의미
# params의 변화값에 대해 GridSearchCV 진행 : GridSearch + cross validation (5(cross) * 5(params))
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
# max_depth = None인 경우, 끝까지 쪼갬
# min_impurity_decrease
# min_samples_leaf = 마지막 노드에 1개가 남을 때까지
# min_samples_split
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [ ]:
# 가장 좋은 성능을 나타내는 모델을 dt에 저장
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# params 변수에서 가장 좋은 성능의 파라미터 값은 무엇인가?
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# CT + Cross validation + GridSearchCV 의 예측 결과
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
# !!! 매우 중요!!!
# 건들일 수 있는 파라미터 3개 => 조합 10*15*10 => 1500개*5개씩 test(cross) = 7500개의 다른 tree를 만들어서 평균냄
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  # 0.0001, 0.0002, ..., 0.0009까지 진행 (10개)
          'max_depth': range(5, 20, 1),                               # 5, 6, ..., 19까지 진행 (15개)
          'min_samples_split': range(2, 100, 10)                      # 2, 12, 22, ..., 92까지 진행 (10개)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [ ]:
# 가장 좋은 성능의 파라미터 표시
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# 성능의 평균
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [ ]:
# 하이퍼파라미터 튜닝 2) RandomSearch : random 하게 학습시키기 때문에 성능이 차이가 존재하지만 평균적으로 낮음
# random search보다 grid search가 성능이 더 좋음, 보편적으로 사용
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 8, 2, 7, 5, 5, 9, 0, 7, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104, 106,  95, 101,  96, 102, 110,  84, 102, 100]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.51345293, 0.08638552, 0.3070773 , 0.05848874, 0.26503564,
       0.21938572, 0.57227675, 0.17134512, 0.37453667, 0.89056224])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [ ]:
# 수업 중에 진행하지 않음, 따로 궁굼한 사람만 실행
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
